In [1]:
%load_ext autotime

In [2]:
# time: 1.16 s

import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from lib.Text_Pre_Processing_in_Python import Preprocess
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA

# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning)

time: 1.84 s


In [3]:
# time: 235 µs

vectorizer_max_features = 1500

time: 277 µs


In [4]:
# time: 6.5 s

all_text = pd.read_csv('archive/sub_reddits.csv', low_memory=False)
all_text['selftext'] = all_text['selftext'].astype(str)

time: 8.92 s


In [5]:
# time: 22min 24s

prep = Preprocess.Preprocess()
sanitized_posts = []

for post in all_text['selftext']:
    document = re.sub(r'https*:\/\/[\w*\-*\.]*[\w\-*\/]*[\.\w]*[^\"\s]*', ' ', str(post))
    document = re.sub(r'\/[\w*\-*\.]*[\w\-*\/]*[\.\w]*[^\"\s]*', ' ', document)
    document = re.sub(r'amp;', '', document)
    document = " ".join(prep.preprocess(document))
    sanitized_posts.append(document)


time: 18min 45s


In [6]:
# time: 191 ms

with open('archive/pre_processed.pkl', 'wb') as picklefile:
    pickle.dump(sanitized_posts, picklefile)

time: 239 ms


In [5]:
# time: 238 ms

with open('archive/pre_processed.pkl', 'rb') as picklefile:
    sanitized_posts_df = pd.DataFrame(pickle.load(picklefile), columns=["posts"])

time: 221 ms


In [6]:
# time: 3.05 s

sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != 'test']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != '']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != 'a']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != '_']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != ' ']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'].str.contains('[^\d^\s]')]
sanitized_posts_df['posts'] = sanitized_posts_df['posts'].astype(str)

sanitized_posts_df.to_csv('archive/sanitized.csv', index=False)

time: 2.93 s


In [ ]:
# time: 703 µs

# Workspace for additional text cleaning

In [7]:
# time: 1.53 s

sanitized_posts_df = pd.read_csv('archive/sanitized.csv')

time: 941 ms


In [8]:
# time: 556 µs

# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(X, km=[], num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    
    if num_clusters == 0:
        plt.scatter(X[:,0], X[:,1], c=color[0], alpha=alpha, s=s)
        
    else:
        for i in range(num_clusters):
            plt.scatter(X[km.labels_==i,0],X[km.labels_==i,1],c = color[i],alpha = alpha,s=s)
            plt.scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)

time: 520 µs


In [9]:
# time: 38.8 s

vec = TfidfVectorizer(stop_words="english")
vec.fit(all_text.selftext.values)
features = vec.transform(all_text.selftext.values)


time: 33 s


In [10]:
# time: 28.9 s

cls = MiniBatchKMeans(n_clusters=5, random_state=0)
cls.fit(features)


MiniBatchKMeans(n_clusters=5, random_state=0)

time: 28 s


In [11]:
# time: 9.15 s

# predict cluster labels for new dataset
cls.predict(features)

# to get cluster labels for the dataset used while
# training the model (used for models that does not
# support prediction on new dataset).
cls.labels_


array([4, 1, 2, ..., 4, 4, 4], dtype=int32)

time: 8.61 s


In [12]:
random_state = 0
# reduce the features to 2D
pca = PCA(n_components=2, random_state=random_state)


time: 346 µs


In [ ]:

reduced_features = pca.fit_transform(features.toarray())


In [ ]:

# reduce the cluster centers to 2D
reduced_cluster_centers = pca.transform(cls.cluster_centers_)

In [ ]:
plt.scatter(reduced_features[:,0], reduced_features[:,1], c=cls.predict(features))
plt.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1], marker='x', s=150, c='b')
